# EJERCICIOS

In [22]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score, mean_squared_error, precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from matplotlib import pyplot as plt
from sklearn.manifold import LocallyLinearEmbedding, MDS, TSNE
from sklearn.pipeline import make_pipeline


## EJERCICIO 1

Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento y un conjunto de prueba (coge las 60.000 primeras instancias para el entrenamiento y las 10.000 restantes para la prueba). 
Entrena un clasificador random forest en el conjunto de datos y cronometra cuánto tarda (puedes usar la anotación %time de jupyter notebook); luego, evalúa el modelo resultante en el conjunto de prueba. 

A continuación, utiliza PCA para reducir la dimensionalidad del conjunto de datos, con un ratio de varianza explicada del 95%. Entrena un nuevo clasificador random forest en el conjunto de datos reducido y comprueba cuánto tarda. ¿Ha sido el entrenamiento mucho más rápido? A continuación, evalúa el clasificador en el conjunto de prueba. ¿Qué tal lo hace en comparación con el clasificador anterior? 

Prueba otra vez con un SGDClassifier, ¿cuánto ayuda ahora el PCA?

In [23]:
mnist = fetch_openml('mnist_784', as_frame=False, parser="auto" )
X_mnist, y_mnist = mnist.data, mnist.target
X_train, y_train = X_mnist[:60_000], y_mnist[:60_000]
X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

In [24]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [25]:
clasificador_RF = RandomForestClassifier(random_state=17)

# Lo entrenamos y cronometramos
print("Entrenando ", clasificador_RF.__class__.__name__)
%time clasificador_RF.fit(X_train, y_train)

Entrenando  RandomForestClassifier
CPU times: total: 15.8 s
Wall time: 33.7 s


RandomForestClassifier(random_state=17)

In [26]:
# Predecimos
y_test_pred_RF= clasificador_RF.predict(X_test)
y_test_pred_RF_proba = clasificador_RF.predict_proba(X_test)  # Para calcular AUC-ROC

# Calcular métricas
exactitud_RF_test = accuracy_score(y_test, y_test_pred_RF)
precision_RF_test = precision_score(y_test, y_test_pred_RF, average='macro')
sensibilidad_RF_test = recall_score(y_test, y_test_pred_RF, average='macro')
f1_RF_test = f1_score(y_test, y_test_pred_RF, average='macro')
roc_auc_RF_test = roc_auc_score(y_test, y_test_pred_RF_proba, multi_class='ovr')

# Mostrar métricas
print("Exactitud:",exactitud_RF_test)
print("Precisión:",precision_RF_test)
print("Sensibilidad:", sensibilidad_RF_test)
print("F1-Score: ",f1_RF_test)
print("Área bajo la curva ROC:",roc_auc_RF_test)

Exactitud: 0.9695
Precisión: 0.9693628551187207
Sensibilidad: 0.9692467563922416
F1-Score:  0.9692823992924285
Área bajo la curva ROC: 0.9991108644118668


In [27]:
# Creamos el reductor PCA a un 95% de varianza y lo aplicamos a los datos mnist
pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

In [15]:
X_train_reduced[0]

array([ 1.23932589e+02,  3.12674262e+02,  2.45140518e+01,  5.55757958e+02,
       -2.72966682e+01,  2.32476284e+02,  2.33995733e+02,  1.59799494e+02,
       -3.63515021e+02,  1.98428077e+02, -1.97497025e+02, -2.54055913e+02,
        1.13496409e+02,  7.49302451e+02, -2.19259554e+02,  4.69324059e+00,
       -3.30151992e+02,  3.09153886e+02,  2.77593870e+02,  1.66332371e+02,
       -2.75418570e+01, -6.43533073e+01,  2.17178328e+02, -2.50244425e+02,
       -4.61660680e+01,  1.00339444e+02,  3.08006769e+02,  1.70105133e+02,
       -1.25514290e+02, -1.23399691e+02, -9.79579471e+01,  3.45751912e+01,
       -1.07484335e+02,  8.45984679e+01, -1.52403071e+02, -2.00802897e+02,
       -6.52408003e+01, -2.00219128e+02, -1.57660546e+02, -3.59742861e+02,
        1.85587109e+02, -1.54450111e+02,  4.02426309e+01,  8.49658546e+01,
        1.23869979e+02, -7.08801921e+00, -1.27540799e+02,  7.64373489e+01,
        3.41432681e+01,  1.99862112e+01, -1.30241511e+02, -1.09555383e+02,
       -1.18375522e+02, -

In [28]:
clasificador_RF2 = RandomForestClassifier(random_state=17)

# Lo entrenamos y cronometramos
print("Entrenando ", clasificador_RF2.__class__.__name__)
%time clasificador_RF2.fit(X_train_reduced, y_train)

Entrenando  RandomForestClassifier
CPU times: total: 47.5 s
Wall time: 1min 36s


RandomForestClassifier(random_state=17)

In [ ]:
# Ha tardado más con el reducido, no deberia, pero por que lo hace escapa a mis conocimientos

In [ ]:
y_test_reduc_RF_pred= clasificador_RF2.predict(X_test_reduced)
y_test_reduc_RF_pred_proba = clasificador_RF2.predict_proba(X_test_reduced)  # Para calcular AUC-ROC

# Calcular métricas
exactitud_test_RF_reduc = accuracy_score(y_test, y_test_reduc_RF_pred)
precision_test_RF_reduc = precision_score(y_test, y_test_reduc_RF_pred, average='macro')
sensibilidad_test_RF_reduc = recall_score(y_test, y_test_reduc_RF_pred, average='macro')
f1_test_RF_reduc = f1_score(y_test, y_test_reduc_RF_pred, average='macro')
roc_auc_test_RF_reduc = roc_auc_score(y_test, y_test_reduc_RF_pred_proba, multi_class='ovr')

# Mostrar métricas
print("Exactitud:",exactitud_test_RF_reduc)
print("Precisión:",precision_test_RF_reduc)
print("Sensibilidad:", sensibilidad_test_RF_reduc)
print("F1-Score: ",f1_test_RF_reduc)
print("Área bajo la curva ROC:",roc_auc_test_RF_reduc)

Exactitud: 0.9487
Precisión: 0.9481653666757032
Sensibilidad: 0.9482132591227828
F1-Score:  0.9481237775653979
Área bajo la curva ROC: 0.9968581824724909


In [ ]:
# Los resultados son algo peores, tal vez si el tiempo de entrenamiento hubiese sido menor seria una perdida aceptable,
# pero visto este caso concreto no hay motivos para usar una reduccion, más tiempo y peores resultados

In [30]:
clasificador_SGD = SGDClassifier(random_state=17)

# Lo entrenamos y cronometramos
print("Entrenando ", clasificador_SGD.__class__.__name__)
%time clasificador_SGD.fit(X_train, y_train)

Entrenando  SGDClassifier
CPU times: total: 47.9 s
Wall time: 2min 2s


SGDClassifier(random_state=17)

In [ ]:
# Predecimos
y_test_pred_SGD= clasificador_SGD.predict(X_test)


# Calcular métricas
exactitud_SDG_test = accuracy_score(y_test, y_test_pred_SGD)
precision_SDG_test = precision_score(y_test, y_test_pred_SGD, average='macro')
sensibilidad_SDG_test = recall_score(y_test, y_test_pred_SGD, average='macro')
f1_SDG_test = f1_score(y_test, y_test_pred_SGD, average='macro')


# Mostrar métricas
print("Exactitud:",exactitud_SDG_test)
print("Precisión:",precision_SDG_test)
print("Sensibilidad:", sensibilidad_SDG_test)
print("F1-Score: ",f1_SDG_test)


Exactitud: 0.8655
Precisión: 0.8809993497655425
Sensibilidad: 0.8658336786181051
F1-Score:  0.8670991513250025


In [ ]:
# Los resultados son peores que con RF, ademas los tiempos son mayores

In [34]:
clasificador_SGD2 = SGDClassifier(random_state=17)

# Lo entrenamos y cronometramos
print("Entrenando ", clasificador_SGD2.__class__.__name__)
%time clasificador_SGD2.fit(X_train_reduced, y_train)

Entrenando  SGDClassifier
CPU times: total: 9.53 s
Wall time: 24 s


SGDClassifier(random_state=17)

In [ ]:
# Los tiempo de SDG si se ven muy reducidos con un reductor PCA

In [35]:
y_test_reduc_SGD_pred=clasificador_SGD2.predict(X_test_reduced)


# Calcular métricas
exactitud_test_SGD_reduc = accuracy_score(y_test, y_test_reduc_SGD_pred)
precision_test_SGD_reduc = precision_score(y_test, y_test_reduc_SGD_pred, average='macro')
sensibilidad_test_SGD_reduc = recall_score(y_test, y_test_reduc_SGD_pred, average='macro')
f1_test_SGD_reduc = f1_score(y_test, y_test_reduc_SGD_pred, average='macro')


# Mostrar métricas
print("Resultados ", clasificador_SGD2.__class__.__name__)
print("Exactitud:",exactitud_test_SGD_reduc)
print("Precisión:",precision_test_SGD_reduc)
print("Sensibilidad:", sensibilidad_test_SGD_reduc)
print("F1-Score: ",f1_test_SGD_reduc)


Resultados  SGDClassifier
Exactitud: 0.8845
Precisión: 0.8834621608276743
Sensibilidad: 0.8833609937693957
F1-Score:  0.8832292457689253


In [ ]:
# PCA ademas de reducir los tiempos haa mejorado los resultados del modelo.

## EJERCICIO 2 (OPCIONAL)

Utiliza t-SNE para reducir las primeras 5.000 imágenes del conjunto de datos MNIST a dos dimensiones (2 columas) 

Representa el resultado utilizando un gráfico de dispersión con 10 colores diferentes para representar la clase objetivo de cada imagen. 

 Prueba a utilizar otros algoritmos de reducción de dimensionalidad como PCA, LLE o MDS y compara las visualizaciones resultantes (la opción de MDS puede que tarde un rato).

 Prueba ahora creando un pipeline que primero aplique PCA y luego MDS (lanzalo con tiempo).